In [9]:
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pandas as pd
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification


/Users/anne-soline.guilbert-ly@dataiku.com/EDF-sentiment-analysis/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', names=['target', 'id', 'date', 'flag', 'user', 'text'])

In [8]:
df_sample = df.sample(n=1000)

In [10]:
# Compute recipe outputs from inputs
# Define the model to use
model_name = "dslim/bert-base-NER"

def perform_ner_inference(model_name, input_df):
    """
    perform_ner_inference performs NER inference on a dataframe using a specified Hugging Face model.
    
    :param model_name: The name of the Hugging Face model to use for NER.
    :param input_df: The input dataframe with at least two columns, document_id and text.
    :return: pd.DataFrame. A dataframe containing the NER results, with at least columns "document_id", "text", and "predicted_labels".
    """
    # Load the pre-trained tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=".cache/")
    model = AutoModelForTokenClassification.from_pretrained(model_name, cache_dir=".cache/")

    # Load the token classification pipeline
    token_classification_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first") # pass device=0 if using gpu

    # Perform token classification on each row of the dataframe
    predicted_labels = []
    for index, row in input_df.iterrows():
        document_id = row["id"]
        text = row["text"]
        results = token_classification_pipeline(text)
        predicted_labels.append(results)
        
    input_df['predicted_labels'] = predicted_labels

    return input_df

document_scored_df = perform_ner_inference(model_name, df_sample)

document_scored_df.head()


NameError: name 'hf_transformers_home_dir' is not defined